# Adding columns

In order to implement Dantzig-Wolfe decomposition we need to be able to add columns to an existing LP.
Gurobi allows us to do that. Let us see how.

First we make the necessary import and build the ``Model`` to which we will add variables.

In [27]:
from gurobipy.gurobipy import Model, GRB, Column
m = Model()

Initially we do not add decision variables. Rather we start from adding an empty objective function.

In [28]:
# We simply pass 0 as the objective function
m.setObjective(0,GRB.MINIMIZE)

In a similar way we will add constraints. In this example we will add two constraints.

In [29]:
# Remember that the method addConstr returns a Constr object
# which represents constraints. Read the documentation 
# https://www.gurobi.com/documentation/8.1/refman/py_model_addconstr.html
# In this way we can assign it to a variable
# and use it when needed. Later we will need this constraint object
# to add columns. 

c1 = m.addConstr(0, GRB.GREATER_EQUAL, 10,"c1")
c2 = m.addConstr(0, GRB.GREATER_EQUAL, 5,"c3")

# This way of adding constraints is preferable to addinng 
# them in an expression form, as follows. For some reasons, 
# the following does not add the desiderd constraints
# c1 = m.addConstr(0 >= 10,"c1")
# c2 = m.addConstr(0 >= 5,"c2")

We will now add columns to this LP. The first step is to build a Column object as explained in the [documentation](https://www.gurobi.com/documentation/8.1/refman/py_column2.html). The second step is to add a decision variable associated to that column as explained in the documentation.

In [30]:
# In order to create a column we need to pass a list of coefficients and a list of constraints.
# The coefficients represent the coefficients with which the column will appear in the respective constraints.
# In this example we will add a column which appears with coefficient 12 in constraint c1 and 5 in constraint c2
col1 = Column([1, 1], [c1, c2])

# We now associate a continuous decision variable to the new column. Let us call it x1.
# The variable will have a coefficient of 19 in the objective function.
# The variable will be associated to column col1 and will thus appear with a coefficient of 
# 12 in constraint c1 and -5 in c2.
m.addVar(lb=0.0, ub=GRB.INFINITY, obj=2, vtype=GRB.CONTINUOUS, name="x1", column=col1)

# We call the method update in order to tell model that we added new stuff.
# This is not necessary if later we solve the problem.
m.update()
# If we now print the model we should see that it has a decision variable
print(m)

<gurobi.Model Continuous instance Unnamed: 2 constrs, 1 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>


Let us now add a two new variables, in a similar way.

In [31]:
col2 = Column([1, -1], [c1, c2])
col3 = Column([1, 0], [c1, c2])
m.addVar(lb=0.0, ub=GRB.INFINITY, obj=3, vtype=GRB.CONTINUOUS, name="x2", column=col2)
m.addVar(lb=0.0, ub=GRB.INFINITY, obj=5, vtype=GRB.CONTINUOUS, name="x3", column=col3)
m.update()
print(m)

<gurobi.Model Continuous instance Unnamed: 2 constrs, 3 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>


Let us now try to solve the problem.

In [38]:
# You can print the problem to a human-readable format as follows.
# See the documentation here https://www.gurobi.com/documentation/8.1/refman/py_model_write.html 
# The file ex.lp will be in the project folder.
m.write("ex.lp")

# We call the method optimize to solve the problem 
m.optimize()

# For each variable we print the name and the optimal value
# See here for getting the list of variables https://www.gurobi.com/documentation/8.1/refman/py_model_getvars.html
# and here for accessing attributes of the variables https://www.gurobi.com/documentation/8.1/refman/py_var_getattr.html
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))
    
# An equivalent way of getting the variables attributes is 
for v in m.getVars():
    print('%s %g' % (v.getAttr(GRB.Attr.VarName), v.getAttr(GRB.Attr.X)))


Optimize a model with 2 rows, 3 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+01]

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.000000000e+01
x1 10
x2 0
x3 0
x1 10
x2 0
x3 0
